In [289]:
Rb = 7.568067737283432
RbO = 8.36675464062834
RbD = 7.568067737283431
Delta_end - 28.84624072309878

from bloqade.analog import load, save
from bloqade.analog.atom_arrangement import Square
import numpy as np
import os
import matplotlib.pyplot as plt

from bokeh.io import output_notebook

In [290]:
from bloqade.analog import load, save
from bloqade.analog.atom_arrangement import Square
import numpy as np
import os
import matplotlib.pyplot as plt

from bokeh.io import output_notebook
output_notebook() 

if not os.path.isdir("data"):
    os.mkdir("data")

# setting the seed
rng = np.random.default_rng(1234)
a=4.5
lattice =Square(4, lattice_spacing=a).apply_defect_density(0.3, rng=rng).remove_vacant_sites()

#durations = [0.3, 1.6, 0.3]

lattice.show()

Loading BokehJS ...

In [291]:
from bloqade.analog import get_capabilities

# get capabilities for Aquila
aquila_capabilities = get_capabilities()

C6=float(aquila_capabilities.capabilities.rydberg.c6_coefficient)
Omega_max= float(aquila_capabilities.capabilities.rydberg.global_.rabi_frequency_max) #2*np.pi*4 #

Rba=np.sqrt(2*np.sqrt(2))
Rb=Rba*a

Delta_end= (C6)/Rb**6 #final detuning 2*np.pi*11

RbO=(C6/Omega_max)**(1/6)
RbD=(C6/Delta_end)**(1/6)

In [292]:
from bloqade.analog import piecewise_linear

T_max=4
sweep_time = 0.8*T_max #time length of the protocol 
ramp_time=0.1*T_max

rabi_amplitude_values = [0.0, Omega_max, Omega_max, 0.0]
detuning_values = [-Delta_end, -Delta_end, Delta_end, Delta_end]
durations = [ramp_time, sweep_time, ramp_time]

RabiProtocol = piecewise_linear(durations,rabi_amplitude_values)
DeltaProtocol = piecewise_linear(durations,detuning_values)

RabiProtocol.show()
DeltaProtocol.show()

In [293]:
mis_udg_program=(
        lattice.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
)

result = mis_udg_program.bloqade.python().run(100,interaction_picture=True)

In [294]:
report = result.report()
report.show()

0.0 13500000.0 0.0 13500000.0


In [295]:
remove_site_indices(n_probable_counts(report))
n_probable_counts(report)

['010111110110']

In [296]:
def n_probable_counts(report, n=1):

        return list(report.counts()[0].keys())[:n]

 
def remove_site_indices(bitstring):

        indices = []

        for i in range(len(bitstring)):

                if bitstring[i] == '0':

                        indices.append(i)

        return indices



In [297]:
most_probable_bitstring = n_probable_counts(report)

indices = remove_site_indices(most_probable_bitstring[0])

children = lattice.children()

locs = [children[i].position for i in range(len(children))]

filled = [True for i in range(len(children))]

for i in indices:

    filled[i] = False

print(most_probable_bitstring)
print(indices)
print(locs)
print(filled)
lattice.show()

['010111110110']
[0, 2, 8, 11]
[(Literal(value=Decimal('0')), Literal(value=Decimal('0'))), (Literal(value=Decimal('0')), Literal(value=Decimal('4.5'))), (Literal(value=Decimal('0')), Literal(value=Decimal('9.0'))), (Literal(value=Decimal('4.5')), Literal(value=Decimal('0'))), (Literal(value=Decimal('4.5')), Literal(value=Decimal('13.5'))), (Literal(value=Decimal('9.0')), Literal(value=Decimal('0'))), (Literal(value=Decimal('9.0')), Literal(value=Decimal('9.0'))), (Literal(value=Decimal('9.0')), Literal(value=Decimal('13.5'))), (Literal(value=Decimal('13.5')), Literal(value=Decimal('0'))), (Literal(value=Decimal('13.5')), Literal(value=Decimal('4.5'))), (Literal(value=Decimal('13.5')), Literal(value=Decimal('9.0'))), (Literal(value=Decimal('13.5')), Literal(value=Decimal('13.5')))]
[False, True, False, True, True, True, True, True, False, True, True, False]


In [298]:
new_lattice = Square(0, lattice_spacing=lattice_spacing).add_position(locs, filled).remove_vacant_sites()

mis_udg_program2=(
        new_lattice.rydberg.rabi.amplitude.uniform.apply(RabiProtocol).detuning.uniform.apply(DeltaProtocol))

result2 = mis_udg_program2.bloqade.python().run(100,interaction_picture=True)

report2 = result2.report()

most_probable_bitstring2 = n_probable_counts(report2)

indices2 = remove_site_indices(most_probable_bitstring2[0])

children2 = new_lattice.children()

locs2 = [children2[i].position for i in range(len(children2))]

filled2 = [True for i in range(len(children2))]

print(most_probable_bitstring2, indices2)

for i in indices2:
    filled2[i] = False

print(filled2)

new_lattice2 = Square(0, lattice_spacing=lattice_spacing).add_position(locs2, filled2).remove_vacant_sites()
new_lattice.show()
new_lattice2.show()


['01001110'] [0, 2, 3, 7]
[False, True, False, False, True, True, True, False]


In [299]:
mis_udg_program3=(
        new_lattice2.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
)

result3 = mis_udg_program3.bloqade.python().run(100,interaction_picture=True)

report3 = result3.report()

most_probable_bitstring3 = n_probable_counts(report3)

indices3 = remove_site_indices(most_probable_bitstring3[0])

children3 = new_lattice2.children()

locs3 = [children3[i].position for i in range(len(children3))]

filled3 = [True for i in range(len(children3))]

print(most_probable_bitstring3, indices3)

for i in indices3:
    filled3[i] = False

print(filled3)

new_lattice3 = Square(0, lattice_spacing=lattice_spacing).add_position(locs3, filled3).remove_vacant_sites()
new_lattice2.show()
new_lattice3.show()


['0100'] [0, 2, 3]
[False, True, False, False]


In [300]:
report3.show()

4500000.0 13500000.0 0.0 13500000.0


In [301]:
mis_udg_program4=(
        new_lattice3.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
)

result4 = mis_udg_program4.bloqade.python().run(100,interaction_picture=True)

report4 = result4.report()

most_probable_bitstring4 = n_probable_counts(report4)

indices4 = remove_site_indices(most_probable_bitstring4[0])

children4 = new_lattice3.children()

locs4 = [children4[i].position for i in range(len(children4))]

filled4 = [True for i in range(len(children4))]

print(most_probable_bitstring4, indices4)

for i in indices4:
    filled4[i] = False

print(filled4)

new_lattice4 = Square(0, lattice_spacing=lattice_spacing).add_position(locs4, filled4).remove_vacant_sites()
new_lattice3.show()
new_lattice4.show()


['0'] [0]
[False]


In [302]:
mis_udg_program=(
        new_lattice2.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
)

result = mis_udg_program.bloqade.python().run(100,interaction_picture=True)

In [303]:
def n_probable_counts(report, n=1):

        return list(report.counts()[0].keys())[:n]

 
def remove_site_indices(bitstring):

        indices = []

        for i in range(len(bitstring)):

                if bitstring[i] == '0':

                        indices.append(i)

        return indices

def post_processing(lattice, result):

        report = result.report()

        most_probable_bitstring = n_probable_counts(report)[0]

        indices = remove_site_indices(most_probable_bitstring)

        children = lattice.children()

        locs = [children[i].position for i in range(len(children))]
        
        print(len(children), len(indices))

        filled = [True for i in range(len(children))]

        for i in indices:

                filled[i] = False

        new_lattice = Square(0, lattice_spacing=lattice_spacing).add_position(locs, filled).remove_vacant_sites()

        return new_lattice, indices

In [304]:
Rb = 7.568067737283432
RbO = 8.36675464062834
RbD = 7.568067737283431
Delta_end - 28.84624072309878

from bloqade.analog import load, save, get_capabilities, piecewise_linear
from bloqade.analog.atom_arrangement import Square

import numpy as np
import os
import matplotlib.pyplot as plt

from bokeh.io import output_notebook

lattice_spacing = 5.0

# get capabilities for Aquila
aquila_capabilities = get_capabilities()

C6=float(aquila_capabilities.capabilities.rydberg.c6_coefficient)
Omega_max= float(aquila_capabilities.capabilities.rydberg.global_.rabi_frequency_max) #2*np.pi*4 #

Rba=np.sqrt(2*np.sqrt(2))
Rb=Rba*a

Delta_end= (C6)/Rb**6 #final detuning 2*np.pi*11

RbO=(C6/Omega_max)**(1/6)
RbD=(C6/Delta_end)**(1/6)


portfolios = []

counter = 0

def mis(lattice):

    lattice.show()

    if not os.path.isdir("data"):
        os.mkdir("data")

    # setting the seed
    rng = np.random.default_rng(1234)

    T_max=4
    sweep_time = 0.8*T_max #time length of the protocol 
    ramp_time=0.1*T_max

    rabi_amplitude_values = [0.0, Omega_max, Omega_max, 0.0]
    detuning_values = [-Delta_end, -Delta_end, Delta_end, Delta_end]
    durations = [ramp_time, sweep_time, ramp_time]

    RabiProtocol = piecewise_linear(durations,rabi_amplitude_values)
    DeltaProtocol = piecewise_linear(durations,detuning_values)

    mis_udg_program=(
        lattice.rydberg.rabi.amplitude.uniform.apply(RabiProtocol).detuning.uniform.apply(DeltaProtocol))

    result = mis_udg_program.bloqade.python().run(100,interaction_picture=True)

    new_lattice, indices = post_processing(lattice, result)

    print(indices)

    if len(indices) ==0:
        portfolios.append(indices)

        return portfolios

    else:
        
        portfolios.append(indices)

        mis(new_lattice)



lattice_init =Square(4, lattice_spacing=lattice_spacing).apply_defect_density(0.3, rng=rng).remove_vacant_sites()

print(mis(lattice_init))

10 4
[0, 1, 5, 9]


6 3
[0, 2, 5]


3 1
[0]


2 1
[1]


1 1
[0]


0 0
[]
None


In [306]:
def n_probable_counts(report, n=1):
    return list(report.counts()[0].keys())[:n]

def remove_site_indices(bitstring):
    return [i for i, bit in enumerate(bitstring) if bit == '0']

def post_processing(result, lattice, lattice_spacing):
    report = result.report()
    bitstring = n_probable_counts(report)[0]
    indices = remove_site_indices(bitstring)
    
    children = lattice.children()
    locs = [children[i].position for i in range(len(children))]
    filled = [i not in indices for i in range(len(children))]
    
    new_lattice = Square(0, lattice_spacing=lattice_spacing).add_position(locs, filled).remove_vacant_sites()
    return new_lattice, indices

def mis(lattice, lattice_spacing, Omega_max, Delta_end, global_indices=None, portfolios=None):
    if portfolios is None:
        portfolios = []
    if global_indices is None:
        global_indices = list(range(len(lattice.children())))

    if len(global_indices) == 0:
        return portfolios

    lattice.show()

    # Setup annealing pulse
    T_max = 4
    sweep_time = 0.8 * T_max
    ramp_time = 0.1 * T_max
    durations = [ramp_time, sweep_time, ramp_time]

    RabiProtocol = piecewise_linear(durations, [0.0, Omega_max, Omega_max, 0.0])
    DeltaProtocol = piecewise_linear(durations, [-Delta_end, -Delta_end, Delta_end, Delta_end])

    program = (
        lattice.rydberg.rabi.amplitude.uniform.apply(RabiProtocol)
        .detuning.uniform.apply(DeltaProtocol)
    )

    result = program.bloqade.python().run(100, interaction_picture=True)
    new_lattice, removed_local_indices = post_processing(result, lattice, lattice_spacing)

    # Map local indices back to global ones
    removed_global_indices = [global_indices[i] for i in removed_local_indices]
    portfolios.append(removed_global_indices)

    # Prepare global indices for next iteration
    new_global_indices = [global_indices[i] for i in range(len(global_indices)) if i not in removed_local_indices]

    if len(new_global_indices) == 0:
        return portfolios
    else:
        return mis(new_lattice, lattice_spacing, Omega_max, Delta_end, new_global_indices, portfolios)


In [307]:
lattice_init = Square(4, lattice_spacing=lattice_spacing).apply_defect_density(0.3, rng=rng).remove_vacant_sites()
portfolios = mis(lattice_init, lattice_spacing, Omega_max, Delta_end)
print("MIS portfolios (global indices):", portfolios)


MIS portfolios (global indices): [[0, 2, 8, 11], [1, 4, 5, 10], [3, 7, 9], [6]]
